### 1) Funcion: describe_df

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from pandas.api.types import is_numeric_dtype

In [2]:
def describe_df(df: pd.DataFrame, decimales: int = 2) -> pd.DataFrame: # con anotaciones para orientar la función
    """
    Devuelve un dataframe resumen con una columna por variable del df original y filas:
    - DATA_TYPE: tipo de dato (dtype)
    - MISSINGS (%): porcentaje de valores nulos
    - UNIQUE_VALUES: número de valores únicos (sin contar NaN)
    - CARDIN (%): porcentaje de cardinalidad = unique_values / n_filas * 100
    """
    if not isinstance(df, pd.DataFrame):
        raise TypeError("df debe ser un pandas DataFrame.")

    n_filas = len(df)

    tipos_dato = df.dtypes.astype(str)
    porcentaje_nulos = (df.isna().mean() * 100).round(decimales)
    valores_unicos = df.nunique(dropna=True)

    if n_filas == 0:
        porcentaje_cardinalidad = pd.Series([0.0] * df.shape[1], index=df.columns)
    else:
        porcentaje_cardinalidad = (valores_unicos / n_filas * 100).round(decimales)

    resumen = pd.DataFrame({
            "DATA_TYPE": tipos_dato,
            "MISSINGS (%)": porcentaje_nulos,
            "UNIQUE_VALUES": valores_unicos,
            "CARDIN (%)": porcentaje_cardinalidad,}).T

    resumen.index.name = "COL_N"
    return resumen

In [3]:
# Reproducible
np.random.seed(42)

n = 300

# 1) Variable continua (muchos valores únicos)
x_continua = np.random.normal(loc=0, scale=1, size=n)

# 2) Variable discreta (enteros con pocos niveles)
x_discreta = np.random.poisson(lam=4, size=n)

# 3) Variable categórica (pocas categorías)
x_categorica = np.random.choice(["Centro", "Salamanca", "Retiro", "Chamartin"], size=n, p=[0.35, 0.25, 0.25, 0.15])

# 4) Variable binaria
x_binaria = np.random.choice([0, 1], size=n, p=[0.6, 0.4])

# 5) Target de regresión (numérico) con relación fuerte con x_continua y moderada con x_discreta
ruido = np.random.normal(loc=0, scale=1.5, size=n)
target = 50 + 12 * x_continua + 2.5 * x_discreta + ruido

# 6) Otra numérica con relación débil (ruido)
x_ruido = np.random.normal(loc=0, scale=1, size=n)

df_sintetico = pd.DataFrame({
    "x_continua": x_continua,
    "x_discreta": x_discreta,
    "x_categorica": x_categorica,
    "x_binaria": x_binaria,
    "x_ruido": x_ruido,
    "target": target
})

# Meter algunos NaN para probar dropna y % missings (aprox 5% en algunas columnas)
for col in ["x_continua", "x_discreta", "x_categorica", "x_ruido"]:
    idx = np.random.choice(df_sintetico.index, size=int(0.05 * n), replace=False)
    df_sintetico.loc[idx, col] = np.nan

df_sintetico.head()

,x_continua,x_discreta,x_categorica,x_binaria,x_ruido,target
0,0.496714,5.0,NaN,0,-0.805109,67.462342
1,-0.138264,8.0,Retiro,0,0.957882,69.650583
2,0.647689,6.0,Chamartin,0,0.779274,72.585336
3,1.523030,5.0,Retiro,0,-0.290575,79.017471
4,-0.234153,NaN,Retiro,0,1.411051,59.669952


In [4]:
df_sintetico.shape

(300, 6)

In [5]:
describe_df(df_sintetico)
describe_df(df_sintetico, decimales=2)

,x_continua,x_discreta,x_categorica,x_binaria,x_ruido,target
COL_N,,,,,,
DATA_TYPE,float64,float64,object,int64,float64,float64
MISSINGS (%),5.0,5.0,5.0,0.0,5.0,0.0
UNIQUE_VALUES,285,11,4,2,285,300
CARDIN (%),95.0,3.67,1.33,0.67,95.0,100.0


### 2) Funcion: tipifica_variables

In [6]:
def tipifica_variables(df: pd.DataFrame, umbral_categoria: int, umbral_continua: float) -> pd.DataFrame:
    """
    Sugiere el tipo de cada variable (columna) de un DataFrame en base a:
    - su cardinalidad (número de valores únicos, sin contar NaN)
    - y su proporción de cardinalidad (valores_unicos / n_filas)

    El usuario DEBE escoger los umbrales, porque son orientativos y dependen del tamaño del dataset y del contexto.

    Reglas de tipificación:
    1) Si la cardinalidad es 2 -> "Binaria"
    2) Si la cardinalidad < umbral_categoria -> "Categórica"
    3) Si la cardinalidad >= umbral_categoria:
        - Si la proporción de cardinalidad >= umbral_continua -> "Numerica Continua"
        - En caso contrario -> "Numerica Discreta"

    Importante:
    - umbral_continua se interpreta como PROPORCIÓN (0 a 1). Ej: 0.8 = 80%
    - No se cuentan NaN como valor único (dropna=True)

    Parametros

    df : pd.DataFrame
        DataFrame de entrada.
    umbral_categoria : int
        Umbral de cardinalidad (entero). Si una variable tiene menos valores únicos que este umbral,
        se sugiere como "Categórica" (salvo que sea binaria).
    umbral_continua : float
        Umbral de proporción (0-1) para decidir si una variable con alta cardinalidad se considera
        "Numerica Continua" (>= umbral_continua) o "Numerica Discreta" (< umbral_continua).

    Returns

    pd.DataFrame
        DataFrame con dos columnas:
        - "nombre_variable": nombre de la columna original
        - "tipo_sugerido": tipo sugerido según las reglas
    """
    if not isinstance(df, pd.DataFrame):
        raise TypeError("df debe ser un pandas DataFrame.")
    if not isinstance(umbral_categoria, int) or umbral_categoria < 2:
        raise ValueError("umbral_categoria debe ser un entero >= 2.")
    if not isinstance(umbral_continua, (int, float)) or not (0 <= umbral_continua <= 1):
        raise ValueError("umbral_continua debe ser un número entre 0 y 1 (proporción).")

    n_filas = len(df)
    cardinalidad = df.nunique(dropna=True)

    if n_filas == 0:
        proporcion_cardinalidad = pd.Series([0.0] * df.shape[1], index=df.columns)
    else:
        proporcion_cardinalidad = cardinalidad / n_filas  # proporción 0-1

    tipos_sugeridos = []
    for col in df.columns:
        card = int(cardinalidad[col])
        prop = float(proporcion_cardinalidad[col])

        if card == 2:
            tipo = "Binaria"
        elif card < umbral_categoria:
            tipo = "Categórica"
        else:
            tipo = "Numerica Continua" if prop >= umbral_continua else "Numerica Discreta"

        tipos_sugeridos.append(tipo)

    return pd.DataFrame({"nombre_variable": df.columns, "tipo_sugerido": tipos_sugeridos})

In [7]:
tipifica_variables(df_sintetico, 4, 0.8)

,nombre_variable,tipo_sugerido
0,x_continua,Numerica Continua
1,x_discreta,Numerica Discreta
2,x_categorica,Numerica Discreta
3,x_binaria,Binaria
4,x_ruido,Numerica Continua
5,target,Numerica Continua


### 3) Funcion: get_features_num_regression

In [ ]:
def get_features_num_regression(df: pd.DataFrame, target_col: str, umbral_corr: float, pvalue: float = None) -> list | None:
   
"""
    Devuelve una lista con las columnas numéricas cuya correlación (Pearson) con `target_col`
    supera en valor absoluto `umbral_corr`.

    Si `pvalue` no es None, además exige significación estadística: p_value <= pvalue
    (equivalente a confianza >= 1 - pvalue).

    Si algún argumento de entrada no es válido, devuelve None y hace print del motivo.
    """

    # ---- Checks de entrada ----
    if not isinstance(df, pd.DataFrame):
        print("Error: df debe ser un pandas DataFrame.")
        return None

    if not isinstance(target_col, str):
        print("Error: target_col debe ser un string con el nombre de una columna.")
        return None

    if target_col not in df.columns:
        print(f"Error: target_col='{target_col}' no existe en el DataFrame.")
        return None

    if not isinstance(umbral_corr, (int, float)) or not (0 <= float(umbral_corr) <= 1):
        print("Error: umbral_corr debe ser un número entre 0 y 1.")
        return None
    umbral_corr = float(umbral_corr)

    if pvalue is not None:
        if not isinstance(pvalue, (int, float)) or not (0 < float(pvalue) <= 1):
            print("Error: pvalue debe ser None o un número en (0, 1].")
            return None
        pvalue = float(pvalue)

    # ---- Check: target numérico y con cardinalidad suficiente ----
    target_serie = df[target_col]
    if not is_numeric_dtype(target_serie):
        print(f"Error: target_col='{target_col}' no es numérica (dtype={target_serie.dtype}).")
        return None

    target_cardinalidad = target_serie.nunique(dropna=True)
    if target_cardinalidad <= 2:
        print(
            f"Error: target_col='{target_col}' tiene cardinalidad {target_cardinalidad}. "
            "Para regresión se espera una variable numérica con más de 2 valores distintos."
        )
        return None

    # ---- Columnas numéricas candidatas (excluyendo target) ----
    columnas_numericas = [columna for columna in df.columns
                          if columna != target_col and is_numeric_dtype(df[columna])]

    if len(columnas_numericas) == 0:
        print("Aviso: no hay columnas numéricas (distintas del target) para evaluar.")
        return []

    columnas_seleccionadas = []

    for columna in columnas_numericas:
        serie_predictora = df[columna]

        # Saltar columnas constantes (o casi vacías)
        if serie_predictora.nunique(dropna=True) <= 1:
            continue

        # Emparejar predictor y target, y quitar filas con NaN en cualquiera de las dos
        datos_pareados = df[[columna, target_col]].dropna()

        if len(datos_pareados) < 3:
            # Muy pocas observaciones para correlación/test
            continue

        # Evitar pearsonr con series constantes tras dropna
        if (
            datos_pareados[columna].nunique(dropna=True) <= 1
            or datos_pareados[target_col].nunique(dropna=True) <= 1
        ):
            continue

        try:
            correlacion_pearson, p_value = pearsonr(
                datos_pareados[columna].astype(float),
                datos_pareados[target_col].astype(float)
            )
        except Exception:
            # Por seguridad: si algo raro pasa (inf, overflow, etc.), saltamos esa columna
            continue

        if np.isnan(correlacion_pearson):
            continue

        # Filtro por correlación y (opcional) por significación estadística
        if abs(correlacion_pearson) > umbral_corr:
            if pvalue is None:
                columnas_seleccionadas.append(columna)
            else:
                if p_value <= pvalue:
                    columnas_seleccionadas.append(columna)

    return columnas_seleccionadas

In [9]:
get_features_num_regression(df_sintetico, target_col="target", umbral_corr=0.3)

['x_continua', 'x_discreta']

In [10]:
df_sintetico

,x_continua,x_discreta,x_categorica,x_binaria,x_ruido,target
0,0.496714,5.0,NaN,0,-0.805109,67.462342
1,-0.138264,8.0,Retiro,0,0.957882,69.650583
2,0.647689,6.0,Chamartin,0,0.779274,72.585336
3,1.523030,5.0,Retiro,0,-0.290575,79.017471
4,-0.234153,NaN,Retiro,0,1.411051,59.669952
...,...,...,...,...,...,...
295,-0.692910,7.0,Centro,0,-1.297169,58.900964
296,0.899600,7.0,Retiro,1,1.015228,75.984827
297,0.307300,4.0,Centro,0,-0.241231,66.662674
298,0.812862,3.0,Retiro,1,0.548879,68.078192
